# 비윤리 문장 AI
- 비윤리 문장 탐지 및 비윤리 문장 상세 안내
    - 상세 안내 카테고리 : ['욕설', '비난', '범죄', '차별', '혐오', '윤리', '성적', '폭력']


1. 데이터셋 수집 : AI-hub의 비윤리 데이터셋을 사용
    1.1 피쳐 및 타겟 : 대화내용, 카테고리
2. 모델 선정 : kobert를 사용
    2.1 선정 이유 : 입력문장의 대부분이 한국어 -> 한국어 데이터셋으로 한국어가 사전학습된 데이터셋을 사용
3. Fine-tuning  
    3.1 파라미터  : 'MAX_LEN = 10 | TRAIN_BATCH_SIZE = 64 | VALID_BATCH_SIZE = 64 | EPOCHS = 10 LEARNING_RATE = 5e-5'
4. 성능 평가
    4.1. 타겟 데이터의 클래스가 불균형함으로 F1score와 acurracy_socre를 둘다 확인 -> 63퍼수준의 정확도가 나옴

In [ ]:
import pandas as pd
import json
from pandas import json_normalize

In [ ]:
# json 형태 파일 pickle로 저장
# with open("/content/drive/MyDrive/tensorflow_project/combined_data.json") as file:

#     data = json.load(file)
# df = json_normalize(data, 'sentences')

In [ ]:
# pickle 불러오기
# import pickle
# with open("/content/drive/MyDrive/tensorflow_project/combined_data.pickle","wb") as fw:
#     pickle.dump(df,fw)
import pickle
with open("/content/drive/MyDrive/tensorflow_project/combined_data.pickle", 'rb') as f:
     data = pickle.load(f)
df = data
# df = json_normalize(data, 'sentences')

In [ ]:
len(df)

363154

## 1. Kobert 환경 세팅

In [ ]:
!pip install mxnet
!pip install gluonnlp==0.8.0
!pip install tqdm pandas
!pip install sentencepiece
!pip install transformers
!pip install torch
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-e8szox_1/kobert-tokenizer_f366e9be7d834474b5697121a703fc33
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-e8szox_1/kobert-tokenizer_f366e9be7d834474b5697121a703fc33
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done


## 2. torch 라이브러리 및 bert, tokenize, vocab 변수 지정

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
device = torch.device("cuda:0")
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

/usr/local/lib/python3.10/dist-packages/mxnet/optimizer/optimizer.py:163: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  warnings.warn('WARNING: New optimizer %s.%s is overriding '


tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [ ]:
## CPU
#device = torch.device("cpu")

## GPU
device = torch.device("cuda:0")

In [ ]:
# 필요한 컬럼만 살리기
df_1 = df[['text','types']]
df_1.sample(n=60)

,text,types
57665,인플레이션 와봤자 돈많은 사람은 살고 서민들만 죽는거지뭐,[IMMORAL_NONE]
288554,그 크기로 했으면 여자가 백퍼 비웃고 놀렸을 듯ㅋㅋ,"[CENSURE, SEXUAL]"
41641,흑형이 우월하긴 하지..,"[DISCRIMINATION, CENSURE]"
152328,너 돌아다니면 땅 울리겠어~ 살 좀 빼.,[CENSURE]
62764,딴 놈한테 다리 벌리느라 못 받은 거 내가 모를 줄 알고?,"[CENSURE, SEXUAL]"
328218,핫팬츠를 입든 말든 자기 마음이지.,[IMMORAL_NONE]
122634,아직 민증에 잉크도 안 마른 아기네. 인생은 어떻게 풀리게 될지 아무도 자신할 수 ...,"[DISCRIMINATION, CENSURE]"
324086,너처럼 능지가 떨어지는 한녀들은 어떻게 처리해야 하냐? ㅋㅋㅋ,"[CENSURE, ABUSE, HATE]"
232226,전라남도 거리두기 3단계 연장이란다,[IMMORAL_NONE]
340640,이번 화 내 최애 악마의 편집 너무 심하게 당했어,[IMMORAL_NONE]


## 3. target data 인코딩
### 3.1 target 데이터가 멀티 라벨 인코딩
    - binary_encoding과 multi_label_encoding로 분리하여 작업

In [ ]:

def binary_encoding(labels, all_classes):
    # labels: 현재 샘플에 속한 라벨 리스트
    # all_classes: 전체 클래스 리스트

    encoding = np.zeros(len(all_classes))
    for label in labels:
        encoding[all_classes.index(label)] = 1

    return encoding

def multi_label_encoding(labels, all_classes):
    # labels: 현재 샘플에 속한 라벨 리스트
    # all_classes: 전체 클래스 리스트

    encoding = np.zeros(len(all_classes))
    for label in labels:
        encoding[all_classes.index(label)] = 1

    return encoding

# 라벨 인코딩 로직
all_classes = ['ABUSE','CENSURE','CRIME','DISCRIMINATION','HATE','IMMORAL_NONE','SEXUAL','VIOLENCE']
df_1['label'] = df_1['types'].apply(lambda x: multi_label_encoding(x,all_classes) if len(x) > 1 else binary_encoding(x, all_classes))

<ipython-input-54-0c92525b1fc0>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['label'] = df_1['types'].apply(lambda x: multi_label_encoding(x,all_classes) if len(x) > 1 else binary_encoding(x, all_classes))


In [ ]:
df_1.label

0         [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1         [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
2         [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
3         [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
4         [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
                            ...                   
363149    [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
363150    [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]
363151    [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
363152    [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]
363153    [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]
Name: label, Length: 363154, dtype: object

## 4. Fine-tuning datase
### 4.1 Fine-tuning시에 맞춰줘야하는 데이터셋 형태로 맞춰줌

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, max_len):
        self.tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
        self.data = dataframe
        self.text = dataframe.text
        self.targets = self.data.label
        self.max_len = max_len


    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=15,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [ ]:
df_1

,text,types,is_immoral,label
0,부랴부랴 왔는데 아무도 안왔네. 시간개념들이 없네,[CENSURE],True,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
1,맞아. 사람들이 진짜 개념이없다,[CENSURE],True,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
2,저렇게는 살지 말아야지,[CENSURE],True,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
3,그러게 게으르고 멍청한 사람들은 맞아야해,[CENSURE],True,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
4,인방 보는 남자는 거르는게 맞다,[CENSURE],True,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
...,...,...,...,...
363149,뷔응신처럼 죽지도 못하면서 괜히 뛰어들기는,[CENSURE],True,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
363150,오죽 하면 그러겠나,[IMMORAL_NONE],False,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]"
363151,너 얼굴 보면 한숨 나온다.,[CENSURE],True,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
363152,신경 끄세요.,[IMMORAL_NONE],False,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]"


## 5. Train, Test dataset 분리

In [ ]:
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(df_1, test_size=0.25, random_state=0)

In [ ]:
# Creating the dataset and dataloader for the neural network

train_size = 0.8
train_dataset=df_1.sample(frac=train_size,random_state=200)
test_dataset=df_1.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)


print("FULL Dataset: {}".format(df_1.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

max_len = 10

training_set = CustomDataset(train_dataset, max_len)
testing_set = CustomDataset(test_dataset, max_len)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


FULL Dataset: (363154, 4)
TRAIN Dataset: (290523, 4)
TEST Dataset: (72631, 4)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


## 6. 파라미터 세팅

In [ ]:

# 파라미터 세팅
MAX_LEN = 10
TRAIN_BATCH_SIZE = 64
VALID_BATCH_SIZE = 64
EPOCHS = 10
LEARNING_RATE = 5e-5

In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0

                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

### 7. 모델 아키텍쳐 구성

In [ ]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, 8)

    def forward(self, ids, mask, token_type_ids):
        _, output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=False)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        return output

model = BERTClass()
model.to(device)

BERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tru

In [ ]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [ ]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

## 8. 학습

In [ ]:
from tqdm import tqdm

def train(epoch):
    model.train()
    for _,data in enumerate(tqdm(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if _%5000==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [ ]:
for epoch in range(EPOCHS):
    train(epoch)

  0%|          | 0/4540 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Epoch: 0, Loss:  0.6905619502067566


  0%|          | 4/4540 [00:00<04:10, 18.11it/s]

Epoch: 1, Loss:  0.2920505404472351


  0%|          | 3/4540 [00:00<03:41, 20.52it/s]

...

  0%|          | 2/4540 [00:00<03:56, 19.21it/s]

Epoch: 8, Loss:  0.09497047960758209


  0%|          | 2/4540 [00:00<04:08, 18.25it/s]

Epoch: 9, Loss:  0.1509692668914795


100%|██████████| 4540/4540 [03:39<00:00, 20.72it/s]


In [ ]:
def validation(epoch):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

## 9. 성능평가
### 9.1 F1-score 사용

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, classification_report
import sklearn.metrics as metrics

for epoch in tqdm(range(EPOCHS)):
    outputs, targets = validation(epoch)
    outputs = np.array(outputs) >= 0.5
    accuracy = metrics.accuracy_score(targets, outputs)
    f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
    f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
    print(f"Accuracy Score = {accuracy}")
    print(f"F1 Score (Micro) = {f1_score_micro}")
    print(f"F1 Score (Macro) = {f1_score_macro}")

  0%|          | 0/3 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
 33%|███▎      | 1/3 [00:29<00:59, 29.97s/it]

Accuracy Score = 0.5106497225702523
F1 Score (Micro) = 0.6264541694385791
F1 Score (Macro) = 0.4045481191233101


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
 67%|██████▋   | 2/3 [00:59<00:29, 29.74s/it]

Accuracy Score = 0.5106497225702523
F1 Score (Micro) = 0.6264541694385791
F1 Score (Macro) = 0.4045481191233101


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████| 3/3 [01:29<00:00, 29.76s/it]

Accuracy Score = 0.5106497225702523
F1 Score (Micro) = 0.6264541694385791
F1 Score (Macro) = 0.4045481191233101


### 10. 모델 저장 및 불러오기

In [ ]:
# import torch

# # # 모델 및 옵티마이저 예시
# # model = ...  # 여기에 저장할 모델을 넣으세요
# # optimizer = ...  # 여기에 저장할 옵티마이저를 넣으세요

# # 저장 경로 지정
# save_path = "/content/drive/MyDrive/tensorflow_project/kobert_model.pth"

# # 모델 및 옵티마이저의 상태를 딕셔너리로 저장
# checkpoint = {
#     'model_state_dict': model.state_dict(),
#     'optimizer_state_dict': optimizer.state_dict(),
# }

# # 저장
# torch.save(checkpoint, save_path)

NameError: ignored

- 실행 시 아래코드만 돌리면됨.

# 실행
## -  여기서 부터만 실행하면됨

# 1. 녹음된 파일로 Speech to text

In [ ]:
# SST 라이브러리
## install
!pip install git+https://github.com/openai/whisper.git

import torch
import librosa
import numpy as np
import soundfile as sf
from scipy.io import wavfile
from IPython.display import Audio
import IPython.display as ipd
import matplotlib.pyplot as plt
from scipy.io import wavfile
from IPython.display import Audio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-z0tp31mh
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-z0tp31mh
  Resolved https://github.com/openai/whisper.git to commit e58f28804528831904c3b6f2c0e473f346223433
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:

# 녹음경로
input_audio_path = '/content/drive/MyDrive/tensorflow_project/wav/이경영 닥쳐 이 병신새끼야 (256  kbps) (shabakngy.com).mp3'  # 한국어1
## import
import whisper
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import librosa
import librosa.display
from pprint import pprint


import IPython.display as ipd
ipd.Audio(input_audio_path) # load a local WAV file

In [ ]:
## load model
model = whisper.load_model("small")
print(f"Model is {'multilingual' if model.is_multilingual else 'English-only'} "
      f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters.")
audio_raw = whisper.load_audio(input_audio_path)
audio = whisper.pad_or_trim(audio_raw)
melspectrogram = whisper.log_mel_spectrogram(audio).to(model.device)
options = whisper.DecodingOptions()
result = whisper.decode(model, melspectrogram, options)

# print the recognized text
input_text = result.text
print(input_text)

Model is multilingual and has 240,582,912 parameters.
닥쳐의 병실이셨게요


In [ ]:
# 비윤리 탐지 라이브러리
!pip install mxnet
!pip install gluonnlp==0.8.0
!pip install tqdm pandas
!pip install sentencepiece
!pip install transformers
!pip install torch
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-7589iv60/kobert-tokenizer_821d894ecf5a4e5a80fbce4ee0265ee5
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-7589iv60/kobert-tokenizer_821d894ecf5a4e5a80fbce4ee0265ee5
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done


In [ ]:
# 파라미터 세팅
MAX_LEN = 10
TRAIN_BATCH_SIZE = 64
VALID_BATCH_SIZE = 64
EPOCHS = 10
LEARNING_RATE = 5e-5

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
device = torch.device("cuda:0")
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, 8)

    def forward(self, ids, mask, token_type_ids):
        _, output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=False)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        return output

model = BERTClass()
model.to(device)

BERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tru

In [ ]:

# 모델 및 옵티마이저의 초기화
model = BERTClass()
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

# 저장된 모델 불러오기
load_path = "/content/drive/MyDrive/tensorflow_project/kobert_model.pth"
checkpoint = torch.load(load_path)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
model.to(device)
# 모델을 evaluation 모드로 설정 (옵션)
model.eval()


BERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tru

## 11. 출력 로직 생성

In [ ]:

from torch.nn.functional import softmax
# input_sentence = input_text
input_sentence = '닥쳐 이 병신 새끼야'
# 토크나이징
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
inputs = tokenizer(input_sentence, return_tensors="pt", padding=True, truncation=True)

# 모델 예측
with torch.no_grad():
    ids = inputs['input_ids'].to(device, dtype = torch.long)
    mask = inputs['attention_mask'].to(device, dtype = torch.long)
    token_type_ids = inputs['token_type_ids'].to(device, dtype = torch.long)
    outputs = model(ids, mask, token_type_ids)

# 소프트맥스 함수를 사용하여 확률로 변환
probs = softmax(outputs.logits, dim=1) if hasattr(outputs, 'logits') else softmax(outputs, dim=1)

 # 모델 예측 결과에서 가장 높은 확률을 가진 클래스의 인덱스 가져오기
predicted_class_index = torch.argmax(probs, dim=1).item()

# 라벨 리스트
labels = ['ABUSE', 'CENSURE', 'CRIME', 'DISCRIMINATION', 'HATE', 'IMMORAL_NONE', 'SEXUAL', 'VIOLENCE']
ko_labels = ['욕설', '비난', '범죄', '차별', '혐오', '윤리', '성적', '폭력']
ko_label_mapping = dict(zip(labels,ko_labels ))
# 예측된 클래스의 인덱스에 해당하는 라벨 가져오기
predicted_label = labels[predicted_class_index]

# 예측 결과 출력

print(predicted_label)
if predicted_label == 'IMMORAL_NONE':
    print(f"비윤리 문장이 감지 되었습니다. : {input_sentence}")
    print('비윤리 문장이 아닙니다')
else:
    print(f"비윤리 문장이 감지 되었습니다. : {input_sentence}")
    print(f"비윤리 문장 유형: {ko_label_mapping.get(predicted_label, 'Unknown')}")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


CENSURE
비윤리 문장이 감지 되었습니다. : 닥쳐 이 병신 새끼야
비윤리 문장 유형: 비난
